In [1]:
import os
os.chdir("../")
os.getcwd()

'/data1/xhuan192/codes/medagents-benchmark'

In [2]:
from pathlib import Path
import json
from dataclasses import dataclass

In [3]:
COST_PER_TOKEN_MAPPING = {
    "gpt-4o": {
        "input": 2.5 / 1_000_000,
        "output": 10 / 1_000_000,
    }
}

@dataclass
class Result:
    accuracy: float
    model_name: str
    num_samples: int
    num_correct: int
    # token usage
    input_tokens: int
    output_tokens: int
    total_tokens: int
    # cost
    input_cost: float
    output_cost: float
    total_cost: float
    # time
    total_time: float
    # average calculations
    # token usage
    avg_input_tokens: float
    avg_output_tokens: float
    avg_tokens: float
    # cost
    avg_input_cost: float
    avg_output_cost: float
    avg_total_cost: float
    # time
    avg_time: float


def parse_result(data, model_name):

    num_correct = 0
    input_tokens = 0
    output_tokens = 0
    total_time = 0

    for sample in data:
        if sample["predicted_answer"].upper() == sample["answer_idx"].upper():
            num_correct += 1
        input_tokens += sample['token_usage']["prompt_tokens"]
        output_tokens += sample['token_usage']["completion_tokens"]
        total_time += sample["time_elapsed"]

    num_samples = len(data)
    accuracy = num_correct / num_samples
    total_tokens = input_tokens + output_tokens

    input_cost = input_tokens * COST_PER_TOKEN_MAPPING[model_name]["input"]
    output_cost = output_tokens * COST_PER_TOKEN_MAPPING[model_name]["output"]
    total_cost = input_cost + output_cost


    # average calculations
    avg_input_tokens = input_tokens / num_samples
    avg_output_tokens = output_tokens / num_samples
    avg_tokens = total_tokens / num_samples

    avg_input_cost = input_cost / num_samples
    avg_output_cost = output_cost / num_samples
    avg_total_cost = total_cost / num_samples

    avg_time = total_time / num_samples

    result_dict =  {
        "accuracy": accuracy,
        "model_name": model_name,
        "num_samples": num_samples,
        "num_correct": num_correct,
        # token usage
        "input_tokens": input_tokens,
        "output_tokens": output_tokens,
        "total_tokens": total_tokens,
        # cost
        "input_cost": input_cost,
        "output_cost": output_cost,
        "total_cost": total_cost,
        # time
        "total_time": total_time,
        # average calculations
        # token usage
        "avg_input_tokens": avg_input_tokens,
        "avg_output_tokens": avg_output_tokens,
        "avg_tokens": avg_tokens,
        # cost
        "avg_input_cost": avg_input_cost,
        "avg_output_cost": avg_output_cost,
        "avg_total_cost": avg_total_cost,
        # time
        "avg_time": avg_time,
    }
    return result_dict


In [4]:
output_dir = Path("outputs")
# get all json files recursively in the output_dir
json_files = list(output_dir.rglob("*.json"))
"""
├── mdagents
│   ├── medbullets
│   │   └── gpt-4o-1120-nofilter-global_medbullets_test_hard_adaptive.json
│   └── pubmedqa
│       └── gpt-4o-1120-nofilter-global_pubmedqa_test_hard_adaptive.json
├── medagents
│   ├── medbullets
│   │   └── gpt-4o-1120-nofilter-global-medbullets-test_hard-syn_verif.json
└── medprompt
    ├── cot
    │   ├── medbullets
    │   │   └── gpt-4o-1120-nofilter-global-medbullets-test_hard-cot.json
"""

method_dataset_result_mapping = {}
model_name = "gpt-4o"
for json_file in json_files:
    # get the method and dataset from the file name
    method, dataset = json_file.parts[-3], json_file.parts[-2]
    model = json_file.parts[-1].split(dataset)[0]

    if method not in method_dataset_result_mapping:
        method_dataset_result_mapping[method] = {}

    # get the json data
    with open(json_file, "r") as f:
        data = json.load(f)
    method_dataset_result_mapping[method][dataset] = parse_result(data, model_name)

In [5]:
def parser_df(method_dataset_result_mapping, key):
    method_dataset_acc_mapping = {}
    for method, dataset_result_mapping in method_dataset_result_mapping.items():
        for dataset, result in dataset_result_mapping.items():
            if method not in method_dataset_acc_mapping:
                method_dataset_acc_mapping[method] = {}
            method_dataset_acc_mapping[method][dataset] = result[key]
    return method_dataset_acc_mapping


In [6]:
import pandas as pd


In [7]:
def display_df(df, key):
    df = pd.DataFrame(parser_df(method_dataset_result_mapping, key))
    df = df[
        ["zero_shot",
        "few_shot",
        "cot",
        "cot_sc",
        "self_refine",
        "multi_persona",
        "medprompt",
        "medagents",
        "mdagents"]
    ]
    df = df.transpose()
    df = df[["medqa", "pubmedqa", "medmcqa", "medbullets", "mmlu", "mmlu-pro", "medexqa", "medxpertqa-r", "medxpertqa-u"]]
    print(f"key: {key}")
    print(f"total: {df.sum().sum()}")

    df["avg"], df["sum"] = df.mean(axis=1), df.sum(axis=1)
    display(df)

In [8]:
for result_key in Result.__dataclass_fields__.keys():
    if result_key in ["model_name"]:
        continue
    display_df(method_dataset_result_mapping, result_key)

key: accuracy
total: 21.433267661997846


,medqa,pubmedqa,medmcqa,medbullets,mmlu,mmlu-pro,medexqa,medxpertqa-r,medxpertqa-u,avg,sum
zero_shot,0.42,0.09,0.28,0.179775,0.287671,0.37,0.15,0.13,0.11,0.224161,2.017447
few_shot,0.35,0.16,0.22,0.258427,0.246575,0.16,0.17,0.14,0.06,0.196111,1.765002
cot,0.55,0.08,0.29,0.314607,0.301370,0.39,0.25,0.19,0.18,0.282886,2.545977
cot_sc,0.52,0.10,0.29,0.292135,0.369863,0.38,0.22,0.15,0.18,0.278000,2.501998
self_refine,0.61,0.15,0.29,0.337079,0.356164,0.37,0.20,0.20,0.21,0.302583,2.723243
multi_persona,0.40,0.11,0.31,0.235955,0.397260,0.45,0.25,0.15,0.12,0.269246,2.423215
medprompt,0.48,0.13,0.30,0.269663,0.397260,0.45,0.20,0.21,0.22,0.295214,2.656923
medagents,0.51,0.16,0.32,0.235955,0.369863,0.38,0.21,0.17,0.16,0.279535,2.515818
mdagents,0.41,0.22,0.31,0.303371,0.260274,0.34,0.22,0.10,0.12,0.253738,2.283645


key: num_samples
total: 7758


,medqa,pubmedqa,medmcqa,medbullets,mmlu,mmlu-pro,medexqa,medxpertqa-r,medxpertqa-u,avg,sum
zero_shot,100,100,100,89,73,100,100,100,100,95.777778,862
few_shot,100,100,100,89,73,100,100,100,100,95.777778,862
cot,100,100,100,89,73,100,100,100,100,95.777778,862
cot_sc,100,100,100,89,73,100,100,100,100,95.777778,862
self_refine,100,100,100,89,73,100,100,100,100,95.777778,862
multi_persona,100,100,100,89,73,100,100,100,100,95.777778,862
medprompt,100,100,100,89,73,100,100,100,100,95.777778,862
medagents,100,100,100,89,73,100,100,100,100,95.777778,862
mdagents,100,100,100,89,73,100,100,100,100,95.777778,862


key: num_correct
total: 2036


...

,medqa,pubmedqa,medmcqa,medbullets,mmlu,mmlu-pro,medexqa,medxpertqa-r,medxpertqa-u,avg,sum
zero_shot,0.000026,0.000024,0.000033,0.000026,0.000024,0.000026,0.000024,0.000029,0.000029,0.000027,0.000241
few_shot,0.002706,0.000503,0.000281,0.003370,0.000742,0.000838,0.000664,0.003130,0.001623,0.001540,0.013857
cot,0.005995,0.004017,0.004003,0.006592,0.004320,0.005479,0.003880,0.007341,0.006569,0.005355,0.048196
cot_sc,0.031015,0.020657,0.020843,0.034429,0.022284,0.029112,0.020479,0.037194,0.033308,0.027702,0.249321
self_refine,0.016606,0.013604,0.012222,0.018776,0.011984,0.013776,0.012010,0.019869,0.016526,0.015042,0.135374
multi_persona,0.051383,0.040318,0.037797,0.056811,0.039398,0.049617,0.036439,0.062256,0.056918,0.047882,0.430938
medprompt,0.019649,0.014220,0.011793,0.022637,0.013404,0.018791,0.013392,0.026416,0.024463,0.018307,0.164764
medagents,0.071546,0.066639,0.062252,0.076986,0.061570,0.063573,0.057047,0.081019,0.072580,0.068135,0.613214
mdagents,0.045109,0.047101,0.010391,0.049885,0.011193,0.019564,0.009297,0.044940,0.037366,0.030538,0.274845


key: avg_total_cost
total: 4.898027625396336


,medqa,pubmedqa,medmcqa,medbullets,mmlu,mmlu-pro,medexqa,medxpertqa-r,medxpertqa-u,avg,sum
zero_shot,0.000768,0.001052,0.000346,0.000838,0.000451,0.000581,0.000360,0.001145,0.000992,0.000726,0.006531
few_shot,0.005556,0.005371,0.001216,0.006290,0.002861,0.003085,0.001942,0.012059,0.010398,0.005420,0.048778
cot,0.006772,0.005081,0.004351,0.007439,0.004782,0.006068,0.004250,0.008492,0.007567,0.006089,0.054801
cot_sc,0.035823,0.027180,0.023074,0.039658,0.025198,0.032793,0.022845,0.044247,0.039440,0.032251,0.290258
self_refine,0.020913,0.018114,0.014508,0.023709,0.014451,0.016703,0.014224,0.025548,0.021152,0.018814,0.169322
multi_persona,0.057318,0.048237,0.040749,0.063234,0.043143,0.054243,0.039542,0.070810,0.064397,0.053519,0.481673
medprompt,0.040877,0.037116,0.022575,0.046491,0.026048,0.035813,0.025722,0.064618,0.060232,0.039943,0.359491
medagents,0.137299,0.135210,0.110920,0.148327,0.112377,0.116957,0.102775,0.159798,0.140566,0.129359,1.164229
mdagents,0.351915,0.586203,0.036886,0.336703,0.046637,0.105661,0.027923,0.438488,0.392530,0.258105,2.322944


key: avg_time
total: 3014.926933091091


,medqa,pubmedqa,medmcqa,medbullets,mmlu,mmlu-pro,medexqa,medxpertqa-r,medxpertqa-u,avg,sum
zero_shot,1.594550,1.200364,1.221147,1.319369,1.256782,1.405757,1.332071,1.303229,1.201106,1.314931,11.834375
few_shot,4.161250,1.824425,1.492047,4.852546,2.007505,2.231216,1.894266,4.972955,3.069181,2.945043,26.505390
cot,7.755876,5.770544,5.623094,8.327458,5.926024,7.281232,5.542304,9.111876,8.332579,7.074554,63.670987
cot_sc,42.611979,30.692409,29.990746,45.911201,31.458270,39.481155,28.852141,46.101323,41.148410,37.360848,336.247634
self_refine,21.883216,18.734010,18.700880,30.797528,31.007710,28.578870,19.923221,30.925171,24.962684,25.057032,225.513291
multi_persona,69.275331,54.875164,51.403859,71.402900,54.418792,66.454602,51.043916,78.360275,71.631152,63.207332,568.865992
medprompt,27.423065,21.865177,19.386590,30.242319,20.521783,26.297989,20.916369,35.358952,31.976376,25.998736,233.988620
medagents,86.033239,79.090347,83.869522,118.260683,89.669220,82.161738,76.703663,100.080238,85.759080,89.069748,801.627731
mdagents,122.749372,181.507588,25.363219,117.420996,24.469562,48.726197,21.595219,112.462730,92.378032,82.963657,746.672913
